In [9]:
import os
from os.path import join
import pickle
import torch
import numpy as np
from pysdf import SDF
import trimesh
from manotorch.manolayer import ManoLayer
from tqdm import *

def convert_to_obj_frame(pc, obj_rot, obj_trans):
    pc = (obj_rot.T @ (pc - obj_trans).T).T
    return pc

def get_close(contact):
        return np.sum(np.abs(contact[np.abs(contact)<0.01]))

datapath = '/root/code/seqs/1205_data/'
obj_path = '/root/code/seqs/object/'
manolayer = ManoLayer(
    mano_assets_root='/root/code/CAMS/data/mano_assets/mano',
    side='right'
)

mano_knuckles_file = '/root/code/CAMS/data/preparation/mano_parts_1.pt' # MANO part file of knuckles
mano_parts_d = torch.load(mano_knuckles_file) # 1552面
verts_part_d = np.zeros(778, dtype=np.int32)
mano_faces = manolayer.get_mano_closed_faces()
for face_id in range(mano_faces.shape[0]):
    verts_part_d[mano_faces[face_id, 0]] = mano_parts_d[face_id]
    verts_part_d[mano_faces[face_id, 1]] = mano_parts_d[face_id]
    verts_part_d[mano_faces[face_id, 2]] = mano_parts_d[face_id]
verts_part_d = torch.from_numpy(verts_part_d)

tips = [15,3,6,12,9]

seqs = sorted(os.listdir(datapath))
grasp_id = []

    


In [15]:
grasp_dict = {}
for seq in tqdm(seqs):
    seq_path = join(datapath,seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    if meta['hand_flag'] == 'right':
        hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
        hand_trans = hand_params[:,:3]
        hand_theta = hand_params[:,3:51]
        mano_beta = hand_params[:,51:]
        active_obj = meta['active_obj']
        obj_mesh_path = join(obj_path,active_obj,'simplified_scan_processed.obj')
        obj_mesh = trimesh.load(obj_mesh_path)
        obj_pose = torch.tensor(np.load(join(seq_path,active_obj+'_pose_trans.npy'))).float()
        obj_rot = obj_pose[:,:,:3]
        obj_trans = obj_pose[:,:,3]
        obj_sdf = SDF(obj_mesh.vertices,obj_mesh.faces)
        for i in range(len(hand_params)):
            thumb_flag = False
            finger_flag = False
            save_flag = False
            mano_output = manolayer(hand_theta[i].unsqueeze(0), mano_beta[i].unsqueeze(0))
            mano_verts = mano_output.verts[0] - mano_output.joints[0, 0] + hand_trans[i]
            mano_verts = convert_to_obj_frame(mano_verts.squeeze(),
                                                obj_rot[i],
                                                obj_trans[i])
            # hand_faces = manolayer.get_mano_closed_faces()
            hand_contact = obj_sdf(mano_verts)
            # print(hand_contact.shape)
            if get_close(hand_contact) > 0:
                for tip in tips:
                    tip_contact = hand_contact[verts_part_d == tip]
                    tip_close = get_close(tip_contact)
                    if tip == 15:
                        if tip_close > 0:
                            thumb_flag = True
                    else:
                        if tip_close > 0 :
                            finger_flag = True
                if thumb_flag and finger_flag:
                    grasp_dict[seq] = i
                    # print(seq,i)
                    break
            else:
                if i == len(hand_params) - 1:
                    print(seq,"failed.")
        # break
    else:
        continue

 13%|█▎        | 185/1378 [19:25<2:13:06,  6.69s/it]

0185 failed.


 25%|██▍       | 338/1378 [38:16<6:04:50, 21.05s/it]

0338 failed.


 25%|██▍       | 341/1378 [39:29<6:27:11, 22.40s/it]

...

 78%|███████▊  | 1072/1378 [2:42:51<37:27,  7.34s/it]  

1072 failed.


 87%|████████▋ | 1194/1378 [2:56:15<57:33, 18.77s/it]  

1194 failed.


100%|██████████| 1378/1378 [3:37:48<00:00,  9.48s/it]  


In [20]:
# grasp_dict = {}
def get_close(contact):
        return np.sum(np.abs(contact[np.abs(contact)<0.03]))
fail_seqs = ['0185','0338','0341','0381','0484','0605','0756','0765','0766','0878','0971','1072','1194']
for seq in tqdm(fail_seqs):
    seq_path = join(datapath,seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    if meta['hand_flag'] == 'right':
        hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
        hand_trans = hand_params[:,:3]
        hand_theta = hand_params[:,3:51]
        mano_beta = hand_params[:,51:]
        active_obj = meta['active_obj']
        print(active_obj)
        obj_mesh_path = join(obj_path,active_obj,'simplified_scan_processed.obj')
        obj_mesh = trimesh.load(obj_mesh_path)
        obj_pose = torch.tensor(np.load(join(seq_path,active_obj+'_pose_trans.npy'))).float()
        obj_rot = obj_pose[:,:,:3]
        obj_trans = obj_pose[:,:,3]
        obj_sdf = SDF(obj_mesh.vertices,obj_mesh.faces)
        for i in range(len(hand_params)):
            thumb_flag = False
            finger_flag = False
            save_flag = False
            mano_output = manolayer(hand_theta[i].unsqueeze(0), mano_beta[i].unsqueeze(0))
            mano_verts = mano_output.verts[0] - mano_output.joints[0, 0] + hand_trans[i]
            mano_verts = convert_to_obj_frame(mano_verts.squeeze(),
                                                obj_rot[i],
                                                obj_trans[i])
            # hand_faces = manolayer.get_mano_closed_faces()
            hand_contact = obj_sdf(mano_verts)
            # print(hand_contact.shape)
            if get_close(hand_contact) > 0:
                for tip in tips:
                    tip_contact = hand_contact[verts_part_d == tip]
                    tip_close = get_close(tip_contact)
                    print(tip,tip_close)
                    if tip == 15:
                        if tip_close > 0:
                            thumb_flag = True
                    else:
                        if tip_close > 0 :
                            finger_flag = True
                if thumb_flag and finger_flag:
                    grasp_dict[seq] = i
                    # print(seq,i)
                    break
            else:
                if i == len(hand_params) - 1:
                    print(seq,"failed.")
        # break
    else:
        continue

  0%|          | 0/13 [00:00<?, ?it/s]

008_chess_whiteking


  8%|▊         | 1/13 [00:36<07:19, 36.62s/it]

0185 failed.
008_chess_whiteking


 15%|█▌        | 2/13 [01:15<06:59, 38.16s/it]

...

0971 failed.
007_pen_5_orange


 92%|█████████▏| 12/13 [06:53<00:29, 29.32s/it]

15 0.029558856
3 0.15925765
6 0.0
12 0.0
9 0.0
002_qua-table-leg


100%|██████████| 13/13 [07:55<00:00, 36.56s/it]

1194 failed.


In [44]:
print(grasp_dict)

{'0001': 14, '0003': 14, '0006': 14, '0007': 13, '0008': 16, '0013': 8, '0019': 10, '0020': 19, '0023': 11, '0024': 10, '0026': 7, '0028': 14, '0031': 15, '0032': 14, '0035': 19, '0037': 20, '0038': 14, '0040': 14, '0042': 18, '0044': 35, '0047': 12, '0048': 18, '0051': 10, '0052': 61, '0055': 18, '0056': 20, '0058': 22, '0061': 16, '0062': 34, '0065': 25, '0067': 34, '0068': 13, '0071': 25, '0072': 27, '0075': 47, '0076': 30, '0079': 44, '0080': 47, '0083': 38, '0084': 38, '0087': 38, '0088': 2, '0091': 43, '0092': 42, '0094': 29, '0096': 36, '0099': 19, '0101': 36, '0102': 3, '0103': 43, '0105': 11, '0106': 8, '0107': 8, '0109': 1, '0111': 62, '0112': 29, '0113': 10, '0114': 46, '0115': 38, '0118': 42, '0119': 33, '0145': 52, '0146': 3, '0147': 72, '0148': 7, '0149': 18, '0150': 9, '0155': 17, '0156': 11, '0160': 48, '0161': 59, '0162': 63, '0163': 43, '0164': 64, '0168': 42, '0169': 33, '0170': 46, '0171': 19, '0172': 75, '0176': 26, '0177': 43, '0178': 34, '0179': 14, '0181': 18, '

In [45]:
goals = np.array(list(grasp_dict.values()))
print(goals.max())

192


In [37]:
goals_seq = list(grasp_dict.keys())
print(goals_seq)

['0001', '0003', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0017', '0019', '0020', '0023', '0024', '0026', '0028', '0031', '0032', '0035', '0037', '0038', '0040', '0042', '0044', '0047', '0048', '0051', '0052', '0055', '0056', '0058', '0061', '0062', '0065', '0067', '0068', '0071', '0072', '0075', '0076', '0079', '0080', '0083', '0084', '0087', '0088', '0091', '0092', '0094', '0096', '0099', '0101', '0102', '0103', '0105', '0106', '0107', '0109', '0111', '0112', '0113', '0114', '0115', '0118', '0119', '0121', '0124', '0127', '0130', '0133', '0140', '0143', '0145', '0146', '0147', '0148', '0149', '0150', '0155', '0156', '0160', '0161', '0162', '0163', '0164', '0168', '0169', '0170', '0171', '0172', '0176', '0177', '0178', '0179', '0181', '0183', '0186', '0187', '0189', '0191', '0192', '0194', '0195', '0196', '0197', '0198', '0200', '0201', '0203', '0204', '0205', '0206', '0207', '0209', '0210', '0211', '0214', '0215', '0219', '0220', '0221',

In [40]:
seq_002 = []
for seq in goals_seq:
    seq_path = join(datapath,seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    if meta['hand_flag'] == 'right':
        hand_params = torch.tensor(np.load(join(seq_path,'mano/poses_right.npy')))
        hand_trans = hand_params[:,:3]
        hand_theta = hand_params[:,3:51]
        mano_beta = hand_params[:,51:]
        active_obj = meta['active_obj']
        # print(active_obj)
        if active_obj.startswith('002_qua') or active_obj.startswith('002_tri'):
            seq_002.append(seq)



In [42]:
# seq_002
for seq in seq_002:
    del grasp_dict[seq]

In [48]:
grasp_dict

{'0001': 14,
 '0003': 14,
 '0006': 14,
 '0007': 13,
 '0008': 16,
 '0013': 8,
 '0019': 10,
 '0020': 19,
 '0023': 11,
 '0024': 10,
 '0026': 7,
 '0028': 14,
 '0031': 15,
 '0032': 14,
 '0035': 19,
 '0037': 20,
 '0038': 14,
 '0040': 14,
 '0042': 18,
 '0044': 35,
 '0047': 12,
 '0048': 18,
 '0051': 10,
 '0052': 61,
 '0055': 18,
 '0056': 20,
 '0058': 22,
 '0061': 16,
 '0062': 34,
 '0065': 25,
 '0067': 34,
 '0068': 13,
 '0071': 25,
 '0072': 27,
 '0075': 47,
 '0076': 30,
 '0079': 44,
 '0080': 47,
 '0083': 38,
 '0084': 38,
 '0087': 38,
 '0088': 2,
 '0091': 43,
 '0092': 42,
 '0094': 29,
 '0096': 36,
 '0099': 19,
 '0101': 36,
 '0102': 3,
 '0103': 43,
 '0105': 11,
 '0106': 8,
 '0107': 8,
 '0109': 1,
 '0111': 62,
 '0112': 29,
 '0113': 10,
 '0114': 46,
 '0115': 38,
 '0118': 42,
 '0119': 33,
 '0145': 52,
 '0146': 3,
 '0147': 72,
 '0148': 7,
 '0149': 18,
 '0150': 9,
 '0155': 17,
 '0156': 11,
 '0160': 48,
 '0161': 59,
 '0162': 63,
 '0163': 43,
 '0164': 64,
 '0168': 42,
 '0169': 33,
 '0170': 46,
 '0171': 

# 去掉失败的部分

In [56]:
for seq,index in grasp_dict.items():
    seq_path = join(datapath,seq)
    meta_path =  join(seq_path,'meta.pkl')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    # print(meta)
    if meta['hand_flag'] == 'right':
        hand_params = np.load(join(seq_path,'mano/poses_right.npy'))
    #     meta['goal_index'] = index
    # with open(meta_path,'wb')as f:
    #     pickle.dump(meta, f)
    break

In [58]:
hand_pose = hand_params[:5,:3]

In [59]:
print(hand_pose)

[[ 0.7396158  -0.9682097  -1.2250646 ]
 [ 0.7460113  -0.9655856  -1.2160189 ]
 [ 0.7521909  -0.9630541  -1.2078402 ]
 [ 0.757985   -0.96067595 -1.2004054 ]
 [ 0.7631883  -0.95849204 -1.1935225 ]]


In [61]:
new_arr = hand_pose[::-1, :]
print(new_arr)

[[ 0.7631883  -0.95849204 -1.1935225 ]
 [ 0.757985   -0.96067595 -1.2004054 ]
 [ 0.7521909  -0.9630541  -1.2078402 ]
 [ 0.7460113  -0.9655856  -1.2160189 ]
 [ 0.7396158  -0.9682097  -1.2250646 ]]


In [63]:
import numpy as np

# 创建一个5x3的数组
array_5x3 = np.array([[1, 2, 3],
                      [4, 5, 6],
                      [7, 8, 9],
                      [10, 11, 12],
                      [13, 14, 15]])

# 指定填充的维度和方式
pad_width = ((0, 5), (0, 0))  # 在第一维度上填充0行，使总行数变为10
array_padded = np.pad(new_arr, pad_width, mode='edge')  # 使用最后一维的边缘值进行填充

print("原始数组:")
print(array_5x3)

print("\n填充后的数组:")
print(array_padded)


原始数组:
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]
 [13 14 15]]

填充后的数组:
[[ 0.7631883  -0.95849204 -1.1935225 ]
 [ 0.757985   -0.96067595 -1.2004054 ]
 [ 0.7521909  -0.9630541  -1.2078402 ]
 [ 0.7460113  -0.9655856  -1.2160189 ]
 [ 0.7396158  -0.9682097  -1.2250646 ]
 [ 0.7396158  -0.9682097  -1.2250646 ]
 [ 0.7396158  -0.9682097  -1.2250646 ]
 [ 0.7396158  -0.9682097  -1.2250646 ]
 [ 0.7396158  -0.9682097  -1.2250646 ]
 [ 0.7396158  -0.9682097  -1.2250646 ]]
